In [ ]:
# Install PySpark (if not already installed)
# !pip install pyspark

# Import SparkSession from pyspark
from pyspark.sql import SparkSession

# Create a local Spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Module5Homework") \
    .getOrCreate()

# Print the Spark version
print("Spark Version:", spark.version)


In [ ]:
# Download the Parquet data (this command runs in the shell)
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet



In [ ]:
# Read the Parquet file into a Spark DataFrame
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")
# Repartition the DataFrame into 4 partitions
df_repart = df.repartition(4)


In [ ]:
# Write the repartitioned DataFrame back to Parquet format
df_repart.write.mode("overwrite").parquet("yellow_tripdata_2024-10_repart.parquet")

In [ ]:
!du -sh yellow_tripdata_2024-10_repart.parquet


In [ ]:
## QUESTION 2

In [ ]:
!ls -lh yellow_tripdata_2024-10_repart.parquet/*.parquet | awk '{sum += $5} END {print sum/NR}'

In [ ]:
import os
import glob

# Directory where the parquet files were written
output_dir = "yellow_tripdata_2024-10_repart.parquet"

# Find all .parquet files (ignore .crc files)
parquet_files = glob.glob(os.path.join(output_dir, "*.parquet"))

# Calculate the total size of the .parquet files
total_size_bytes = 0
for f in parquet_files:
    total_size_bytes += os.path.getsize(f)

# Compute average size in MB
if len(parquet_files) > 0:
    avg_size_mb = total_size_bytes / len(parquet_files) / (1024 * 1024)
    print(f"Average Parquet file size: {avg_size_mb:.2f} MB")
else:
    print("No .parquet files found.")


In [ ]:
## question 3


In [ ]:
from pyspark.sql.functions import to_date, col

# Let's assume 'df' is the DataFrame containing your taxi data
columns_lower = [c.lower() for c in df.columns]
pickup_candidates = [df.columns[i] for i, c in enumerate(columns_lower) if "pickup" in c]

if len(pickup_candidates) == 0:
    raise ValueError("No column found with the word 'pickup' in its name.")
elif len(pickup_candidates) > 1:
    raise ValueError(f"Multiple columns found containing 'pickup': {pickup_candidates}")
else:
    pickup_col = pickup_candidates[0]
    print(f"Using pickup column: {pickup_col}")

# Now filter for October 15th (2024-10-15)
df_oct15 = df.filter(to_date(col(pickup_col)) == "2024-10-15")

# Count records
trip_count_oct15 = df_oct15.count()
print("Number of trips on October 15th:", trip_count_oct15)


In [ ]:
columns_lower


In [ ]:
#limit to those that finish same day

In [ ]:
from pyspark.sql.functions import to_date, col

# Assume your DataFrame is called df

# 1) Dynamically detect the pickup column name by searching for "pickup" in df.columns
columns_lower = [c.lower() for c in df.columns]
pickup_candidates = [df.columns[i] for i, c in enumerate(columns_lower) if "pickup" in c]

if len(pickup_candidates) == 0:
    raise ValueError("No column found with the word 'pickup' in its name.")
elif len(pickup_candidates) > 1:
    raise ValueError(f"Multiple columns found containing 'pickup': {pickup_candidates}")
else:
    pickup_col = pickup_candidates[0]
    print(f"Using pickup column: {pickup_col}")

# 2) Filter for trips that started AND ended on 2024-10-15
df_oct15 = df.filter(
    (to_date(col(pickup_col)) == "2024-10-15") &
    (to_date(col("tpep_dropoff_datetime")) == "2024-10-15")
)

# # 3) Show the filtered records (or count them)
# df_oct15.show()  # or df_oct15.count()


In [ ]:
# Count records
trip_count_oct15 = df_oct15.count()
print("Number of trips on October 15th:", trip_count_oct15)


In [ ]:
## question 4 Longest trip in hours

In [ ]:
from pyspark.sql.functions import unix_timestamp, max as spark_max

# Create a new column for trip duration in hours
df_duration = df.withColumn("duration_hours", 
                            (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 3600)

# Find the longest trip (maximum duration)
max_duration = df_duration.select(spark_max("duration_hours")).collect()[0][0]
print("Longest trip duration (hours):", max_duration)


In [ ]:
## question 6

In [ ]:
# Download the taxi zone lookup CSV file
# !wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

# Load the CSV into a Spark DataFrame
zone_df = spark.read.csv("taxi_zone_lookup.csv", header=True, inferSchema=True)
zone_df.createOrReplaceTempView("zones")

# Create or use a temporary view for the Yellow taxi data if not already done
df.createOrReplaceTempView("yellow")

# Use a Spark SQL query to join the data, count trips per zone, and select the least frequent one.
least_frequent_zone = spark.sql("""
SELECT z.zone, COUNT(*) as trip_count
FROM yellow y
JOIN zones z ON y.PULocationID = z.LocationID
GROUP BY z.zone
ORDER BY trip_count ASC
""")
#LIMIT 1

least_frequent_zone.show()
